In [1]:
import os, sys, time, glob
import warnings
import re
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import pandana as pdna
pd.options.mode.chained_assignment = None
import h5py
sys.path.append(os.path.join(os.getcwd(),'../../../..'))
# from input_configuration import base_year
import toml
config = toml.load(os.path.join(os.getcwd(),'../../../../configuration/input_configuration.toml'))
summary_config = toml.load(os.path.join(os.getcwd(),'../../../../configuration/summary_configuration.toml'))

%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
output_path = os.path.join(summary_config['sc_run_path'], summary_config["output_folder"])
survey_path = os.path.join(summary_config['sc_run_path'], summary_config["survey_folder"])
output_dir = os.path.join(output_path,'compare\RTP\mode_share')

# Write out CSVs with a numeric value prefix to help organize outputs 
incr = 1

In [3]:
pd.options.display.float_format = '{:0,.0f}'.format

In [4]:
pd.options.display.float_format = '{:0,.1%}'.format

mode_share_county = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_county.csv'))
df_person = pd.read_csv(os.path.join(output_path,'agg\dash\person_geog.csv'))

### Region

In [5]:
result_df = pd.DataFrame()

df_trip = mode_share_county.copy()
df_trip.loc[df_trip['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
df_trip.loc[df_trip['dpurp'] == 'Work', 'trip_type'] = 'Work'
for purp in ['All Trips','Work','Non-Work']:
    if purp != 'All Trips':
        df = df_trip[df_trip['trip_type'] == purp]
    else:
        df = df_trip.copy()
    _df = df.groupby('mode').sum()[['trexpfac']]/df['trexpfac'].sum()
    _df['purp'] = purp
    result_df = result_df.append(_df)
_df = result_df.pivot_table(columns='purp', index='mode', values='trexpfac', aggfunc='sum')
_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_region.csv')); incr+=1
_df = _df.reset_index()
_df.rename(columns={'mode': 'Mode'}, inplace=True)
_df = _df.rename_axis(None, axis=1)
HTML(_df.to_html(index=False))

Mode,All Trips,Non-Work,Work
Bike,1.5%,1.5%,1.4%
HOV2,22.4%,24.3%,9.6%
HOV3+,14.6%,15.9%,6.0%
SOV,44.7%,40.7%,71.6%
School Bus,1.5%,1.8%,0.0%
TNC,0.3%,0.3%,0.6%
Transit,2.1%,2.1%,2.3%
Walk,12.8%,13.5%,8.5%


### County

In [6]:
def calculate_mode_share(df, geography, purp=None):
    
    df.loc[df['dpurp'] != 'Work', 'trip_type'] = 'Non-Work'
    df.loc[df['dpurp'] == 'Work', 'trip_type'] = 'Work'
    
    if purp:
        df = df[df['trip_type'] == purp]
    
    _df = df.pivot_table(index='mode', columns=geography, values='trexpfac', aggfunc='sum')
    
    for col in _df:
        _df[col] = _df[col]/_df[col].sum()
    return _df.T

In [7]:
df = mode_share_county.copy()
df = calculate_mode_share(df, 'hh_county')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_county.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_county': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
King,1.6%,21.5%,13.8%,43.9%,1.4%,0.4%,3.3%,14.1%
Kitsap,1.3%,22.8%,14.5%,45.0%,1.4%,0.2%,1.2%,13.7%
Outside Region,NaN,20.0%,40.0%,40.0%,NaN,NaN,NaN,NaN
Pierce,1.4%,23.9%,16.0%,45.0%,1.8%,0.2%,0.7%,11.0%
Snohomish,1.3%,23.0%,15.5%,46.6%,1.6%,0.2%,0.8%,11.0%


In [8]:
df = mode_share_county.copy()
df = calculate_mode_share(df, 'hh_county', 'Non-Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_county_nonwork.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_county': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
King,1.5%,23.5%,15.1%,40.0%,1.7%,0.4%,3.3%,14.6%
Kitsap,1.3%,24.4%,15.5%,41.6%,1.6%,0.2%,0.8%,14.5%
Outside Region,NaN,22.2%,44.4%,33.3%,NaN,NaN,NaN,NaN
Pierce,1.4%,25.7%,17.2%,41.0%,2.0%,0.2%,0.7%,11.7%
Snohomish,1.4%,24.8%,16.8%,42.3%,1.8%,0.2%,0.8%,11.9%


In [9]:
df = mode_share_county.copy()
df = calculate_mode_share(df, 'hh_county', 'Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_county_work.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_county': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
King,1.7%,9.3%,5.7%,68.5%,0.0%,0.8%,3.0%,11.1%
Kitsap,1.0%,9.6%,5.9%,72.3%,0.0%,0.2%,4.1%,6.9%
Outside Region,NaN,NaN,NaN,100.0%,NaN,NaN,NaN,NaN
Pierce,1.0%,10.1%,6.3%,76.0%,0.0%,0.3%,1.1%,5.1%
Snohomish,0.9%,10.2%,6.5%,76.9%,0.0%,0.2%,0.6%,4.6%


## Regional Growth Center

In [10]:
mode_share_rgc = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_rgc.csv'))
df = mode_share_rgc.copy()

# Create results that sum up trips for households in all RGCs
_df = df[df['hh_rgc'] != 'Not in RGC'].groupby(['dpurp','mode']).sum()[['trexpfac']].reset_index()
_df['hh_rgc'] = 'All RGCs'
df = df.append(_df)
df = calculate_mode_share(df, 'hh_rgc')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rgc.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rgc': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
All RGCs,3.0%,12.3%,5.8%,30.6%,0.5%,1.6%,6.7%,39.4%
Auburn,2.0%,21.0%,12.3%,39.7%,0.8%,0.6%,3.2%,20.5%
Bellevue,2.5%,14.0%,5.9%,34.1%,0.4%,0.8%,4.6%,37.8%
Bothell Canyon Park,1.9%,23.1%,10.1%,49.7%,1.6%,0.5%,0.9%,12.3%
Bremerton,2.4%,16.2%,7.8%,36.2%,0.4%,0.5%,2.0%,34.4%
Burien,1.5%,18.9%,9.9%,45.9%,0.7%,0.4%,3.7%,19.0%
Everett,2.2%,17.0%,9.1%,42.7%,0.7%,0.5%,2.8%,25.0%
Federal Way,2.0%,22.2%,15.4%,38.9%,1.9%,0.1%,2.1%,17.4%
Greater Downtown Kirkland,1.7%,20.8%,9.3%,49.6%,0.8%,0.5%,4.2%,13.0%
Kent,2.4%,19.6%,12.9%,37.3%,1.5%,0.4%,4.1%,21.7%


In [11]:
df = mode_share_rgc.copy()

# Create results that sum up trips for households in all RGCs
_df = df[df['hh_rgc'] != 'Not in RGC'].groupby(['dpurp','mode']).sum()[['trexpfac']].reset_index()
_df['hh_rgc'] = 'All RGCs'
df = df.append(_df)

df = calculate_mode_share(df, 'hh_rgc', 'Non-Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rgc_nonwork.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rgc': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
All RGCs,2.9%,13.7%,6.5%,28.2%,0.6%,1.3%,7.2%,39.7%
Auburn,1.9%,22.4%,13.0%,36.9%,0.9%,0.6%,3.0%,21.3%
Bellevue,2.3%,15.6%,6.5%,31.1%,0.5%,0.7%,5.1%,38.3%
Bothell Canyon Park,1.8%,25.3%,11.0%,45.4%,1.8%,0.4%,0.9%,13.4%
Bremerton,2.3%,17.5%,8.4%,33.0%,0.4%,0.5%,1.4%,36.7%
Burien,1.4%,20.5%,10.4%,41.5%,0.8%,0.4%,4.0%,21.0%
Everett,2.2%,18.5%,10.1%,38.9%,0.8%,0.5%,3.1%,26.0%
Federal Way,1.9%,23.8%,16.5%,35.1%,2.1%,0.1%,2.3%,18.3%
Greater Downtown Kirkland,1.6%,22.9%,10.3%,46.1%,0.9%,0.5%,4.2%,13.5%
Kent,2.2%,20.8%,14.0%,33.6%,1.7%,0.5%,4.2%,23.0%


In [12]:
df = mode_share_rgc.copy()

# Create results that sum up trips for households in all RGCs
_df = df[df['hh_rgc'] != 'Not in RGC'].groupby(['dpurp','mode']).sum()[['trexpfac']].reset_index()
_df['hh_rgc'] = 'All RGCs'
df = df.append(_df)

df = calculate_mode_share(df, 'hh_rgc', 'Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rgc_work.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rgc': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
All RGCs,3.3%,5.6%,2.5%,43.0%,0.0%,2.7%,4.5%,38.4%
Auburn,2.9%,8.7%,6.2%,63.1%,NaN,0.9%,4.6%,13.7%
Bellevue,3.2%,5.9%,2.7%,49.6%,NaN,1.4%,1.9%,35.4%
Bothell Canyon Park,2.5%,10.1%,4.4%,75.7%,NaN,0.8%,1.1%,5.4%
Bremerton,3.4%,7.3%,3.9%,59.8%,NaN,0.7%,6.3%,18.5%
Burien,2.1%,9.3%,6.7%,71.9%,NaN,0.5%,2.4%,7.1%
Everett,2.5%,7.1%,3.1%,67.4%,NaN,0.4%,1.0%,18.6%
Federal Way,3.0%,8.5%,5.9%,72.5%,NaN,NaN,0.4%,9.7%
Greater Downtown Kirkland,2.1%,8.4%,3.7%,70.9%,NaN,0.5%,4.2%,10.1%
Kent,3.6%,10.8%,4.4%,66.1%,NaN,NaN,3.7%,11.4%


## Regional Geography

In [13]:
mode_share_rg = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_rg.csv'))
df = mode_share_rg.copy()

df = calculate_mode_share(df, 'hh_rg_proposed')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rg.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rg_proposed': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
CitiesTowns,1.2%,23.6%,17.0%,45.1%,2.0%,0.2%,0.5%,10.5%
Core,1.3%,23.6%,15.5%,46.1%,1.6%,0.2%,1.7%,9.9%
HCT,1.3%,23.3%,15.6%,46.0%,1.7%,0.2%,1.3%,10.6%
Metro,2.0%,19.1%,11.3%,40.8%,1.1%,0.6%,4.4%,20.6%
UU,1.2%,24.5%,17.9%,44.1%,2.0%,0.1%,0.4%,9.7%


In [14]:
df = mode_share_rg.copy()
df = calculate_mode_share(df, 'hh_rg_proposed', 'Non-Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rg_nonwork.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rg_proposed': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
CitiesTowns,1.2%,25.3%,18.3%,40.8%,2.3%,0.2%,0.5%,11.4%
Core,1.4%,25.5%,16.8%,41.9%,1.9%,0.2%,1.7%,10.6%
HCT,1.3%,25.2%,16.9%,41.8%,2.0%,0.2%,1.2%,11.4%
Metro,2.0%,20.9%,12.4%,37.0%,1.3%,0.5%,4.5%,21.4%
UU,1.3%,26.3%,19.3%,39.9%,2.3%,0.1%,0.4%,10.5%


In [15]:
df = mode_share_rg.copy()
df = calculate_mode_share(df, 'hh_rg_proposed', 'Work')
df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_rg_work.csv')); incr+=1
df = df.reset_index()
df.rename(columns={'hh_rg_proposed': 'Geography'}, inplace=True)
df = df.rename_axis(None, axis=1)
HTML(df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
CitiesTowns,0.6%,10.6%,7.0%,76.4%,0.0%,0.2%,0.8%,4.3%
Core,1.2%,10.2%,6.5%,74.5%,0.0%,0.3%,1.9%,5.3%
HCT,0.9%,10.2%,6.5%,75.8%,0.0%,0.3%,1.6%,4.7%
Metro,2.3%,8.5%,4.8%,63.3%,0.0%,1.1%,3.8%,16.2%
UU,0.7%,10.4%,7.0%,77.2%,0.0%,0.2%,0.9%,3.5%


## Equity Geography

In [16]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_'+col+'.csv'))
        df = df[df[col] == 1]
        if len(df) > 0:
            df = calculate_mode_share(df, col)
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_youth_reg,1.2%,23.9%,16.7%,45.3%,2.0%,0.2%,1.2%,9.5%
hh_elderly_reg,1.3%,22.9%,14.4%,45.9%,1.4%,0.3%,1.9%,11.8%
hh_elderly_50,0.6%,26.9%,10.7%,57.1%,0.7%,0.2%,0.2%,3.7%
hh_english_reg,1.4%,22.9%,15.3%,44.5%,1.6%,0.3%,2.3%,11.6%
hh_racial_reg,1.6%,22.4%,14.9%,43.1%,1.6%,0.4%,2.5%,13.6%
hh_racial_50,1.6%,22.4%,15.0%,42.8%,1.6%,0.4%,2.7%,13.6%
hh_poverty_reg,1.6%,22.2%,14.4%,43.1%,1.5%,0.4%,2.5%,14.2%
hh_poverty_50,2.2%,20.6%,13.7%,35.8%,1.4%,0.5%,5.9%,19.9%
hh_disability_reg,1.5%,22.4%,14.1%,44.3%,1.4%,0.4%,2.1%,13.9%


In [17]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_'+col+'.csv'))
        df = df[df[col] == 0]
        if len(df) > 0:
            df = calculate_mode_share(df, col)
            df['source'] = 'NOT in '+col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
NOT in hh_youth_reg,1.7%,20.9%,12.6%,44.1%,1.1%,0.5%,3.0%,16.1%
NOT in hh_youth_50,1.5%,22.4%,14.6%,44.7%,1.5%,0.3%,2.1%,12.8%
NOT in hh_elderly_reg,1.6%,21.9%,14.8%,43.6%,1.6%,0.3%,2.3%,13.8%
NOT in hh_elderly_50,1.5%,22.4%,14.6%,44.7%,1.5%,0.3%,2.1%,12.8%
NOT in hh_english_reg,1.5%,22.0%,14.2%,44.8%,1.5%,0.3%,2.0%,13.6%
NOT in hh_english_50,1.5%,22.4%,14.6%,44.7%,1.5%,0.3%,2.1%,12.8%
NOT in hh_racial_reg,1.4%,22.4%,14.4%,46.1%,1.5%,0.3%,1.8%,12.2%
NOT in hh_racial_50,1.4%,22.4%,14.4%,45.6%,1.5%,0.3%,1.9%,12.5%
NOT in hh_poverty_reg,1.4%,22.5%,14.7%,45.7%,1.5%,0.3%,1.9%,12.0%
NOT in hh_poverty_50,1.4%,22.4%,14.6%,44.9%,1.5%,0.3%,2.1%,12.7%


In [18]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_'+col+'.csv'))
        df = df[df[col] == 1]
        if len(df) > 0:
            df = calculate_mode_share(df, col, 'Non-Work')
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity_nonwork.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_youth_reg,1.3%,25.8%,18.1%,41.1%,2.2%,0.2%,1.2%,10.2%
hh_elderly_reg,1.3%,24.8%,15.5%,42.2%,1.6%,0.3%,1.9%,12.3%
hh_elderly_50,0.6%,28.0%,11.0%,55.7%,0.7%,0.1%,0.2%,3.7%
hh_english_reg,1.5%,24.9%,16.7%,40.4%,1.9%,0.3%,2.3%,12.2%
hh_racial_reg,1.6%,24.3%,16.2%,39.1%,1.8%,0.3%,2.5%,14.1%
hh_racial_50,1.6%,24.3%,16.3%,38.7%,1.9%,0.3%,2.7%,14.1%
hh_poverty_reg,1.6%,24.0%,15.6%,39.2%,1.7%,0.3%,2.5%,14.9%
hh_poverty_50,2.1%,22.2%,14.8%,32.3%,1.5%,0.5%,6.0%,20.6%
hh_disability_reg,1.5%,24.2%,15.3%,40.5%,1.6%,0.3%,2.0%,14.5%


In [19]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_'+col+'.csv'))
        df = df[df[col] == 0]
        if len(df) > 0:
            df = calculate_mode_share(df, col, 'Non-Work')
            df['source'] = 'NOT in '+col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity_nonwork.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
NOT in hh_youth_reg,1.7%,22.8%,13.7%,40.4%,1.3%,0.4%,3.0%,16.7%
NOT in hh_youth_50,1.5%,24.3%,15.9%,40.7%,1.8%,0.3%,2.1%,13.5%
NOT in hh_elderly_reg,1.6%,23.8%,16.2%,39.4%,1.9%,0.3%,2.3%,14.5%
NOT in hh_elderly_50,1.5%,24.3%,15.9%,40.7%,1.8%,0.3%,2.1%,13.5%
NOT in hh_english_reg,1.5%,23.9%,15.4%,41.0%,1.7%,0.3%,2.0%,14.3%
NOT in hh_english_50,1.5%,24.3%,15.9%,40.7%,1.8%,0.3%,2.1%,13.5%
NOT in hh_racial_reg,1.4%,24.3%,15.6%,42.1%,1.7%,0.3%,1.8%,12.9%
NOT in hh_racial_50,1.4%,24.3%,15.7%,41.7%,1.7%,0.3%,1.8%,13.2%
NOT in hh_poverty_reg,1.4%,24.4%,16.1%,41.7%,1.8%,0.3%,1.9%,12.5%
NOT in hh_poverty_50,1.5%,24.3%,15.9%,40.9%,1.8%,0.3%,2.0%,13.3%


In [20]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_'+col+'.csv'))
        df = df[df[col] == 1]
        if len(df) > 0:
            df = calculate_mode_share(df, col, 'Work')
            df['source'] = col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity_work.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))

Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_youth_reg,0.9%,10.4%,6.8%,75.7%,0.0%,0.3%,1.4%,4.6%
hh_elderly_reg,1.2%,9.7%,6.1%,72.8%,0.0%,0.5%,2.2%,7.6%
hh_elderly_50,0.7%,9.0%,6.5%,79.9%,NaN,0.2%,0.6%,3.0%
hh_english_reg,1.4%,10.0%,6.3%,72.2%,0.0%,0.5%,2.0%,7.6%
hh_racial_reg,1.5%,9.6%,6.0%,69.7%,0.0%,0.7%,2.2%,10.3%
hh_racial_50,1.6%,9.7%,6.1%,69.2%,0.0%,0.7%,2.2%,10.4%
hh_poverty_reg,1.6%,9.7%,6.0%,70.4%,0.0%,0.6%,2.5%,9.3%
hh_poverty_50,2.5%,8.5%,5.2%,63.5%,0.0%,0.8%,5.5%,14.0%
hh_disability_reg,1.4%,9.6%,5.9%,71.2%,0.0%,0.6%,2.1%,9.1%


In [21]:
result_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        col = equity_geog+geog_type
        df = pd.read_csv(os.path.join(output_path,'agg\dash\mode_share_'+col+'.csv'))
        df = df[df[col] == 0]
        if len(df) > 0:
            df = calculate_mode_share(df, col, 'Work')
            df['source'] = 'NOT in '+col
            result_df = result_df.append(df)
result_df.index = result_df.source
result_df.drop('source', axis=1, inplace=True)
result_df.to_csv(os.path.join(output_dir,str(incr)+'_mode_share_equity_work.csv')); incr+=1
result_df = result_df.reset_index()
result_df = result_df.rename_axis(None, axis=1)
result_df.rename(columns={'source': 'Geography'}, inplace=True)
HTML(result_df.to_html(index=False))


Geography,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
NOT in hh_youth_reg,1.8%,9.0%,5.3%,68.2%,0.0%,0.8%,3.0%,12.0%
NOT in hh_youth_50,1.4%,9.6%,6.0%,71.6%,0.0%,0.6%,2.3%,8.5%
NOT in hh_elderly_reg,1.5%,9.6%,5.9%,70.7%,0.0%,0.6%,2.3%,9.3%
NOT in hh_elderly_50,1.4%,9.6%,6.0%,71.6%,0.0%,0.6%,2.3%,8.6%
NOT in hh_english_reg,1.4%,9.4%,5.8%,71.3%,0.0%,0.6%,2.4%,9.1%
NOT in hh_english_50,1.4%,9.6%,6.0%,71.6%,0.0%,0.6%,2.3%,8.5%
NOT in hh_racial_reg,1.2%,9.7%,5.9%,73.4%,0.0%,0.4%,2.4%,7.0%
NOT in hh_racial_50,1.3%,9.6%,5.9%,72.8%,0.0%,0.5%,2.3%,7.6%
NOT in hh_poverty_reg,1.3%,9.6%,6.0%,72.4%,0.0%,0.5%,2.1%,8.1%
NOT in hh_poverty_50,1.4%,9.7%,6.0%,71.8%,0.0%,0.5%,2.2%,8.5%
